In [5]:
import pandas as pd
import numpy as np

In [6]:
# Create a Simple, Clean Synthetic Dataset
np.random.seed(41)
n=1000

df=pd.DataFrame({
    "Student_ID": np.arange(101,101+n),
    "Age": np.random.randint(18,25,size=n),
    "Marks": np.random.randint(50,100,size=n),
    "Department": np.random.choice(["AIML", "CSE", "ECE", "MECH","AUTOMATION"],size=n)
})


In [8]:
#Introduce Missing Values in Age, Marks, Department
for col in ["Age","Marks","Department"]:
    df.loc[df.sample(frac=0.5).index, col] = np.nan

## Duplicate records/rows
df = pd.concat([df, df.sample(20)], ignore_index=True)

#Introduce Invalid ages
df.loc[df.sample(10).index, "Age"] = np.random.choice([-5, -10, -15], size=10)

#Introduce outliers
df.loc[df.sample(10).index, "Marks"] = np.random.choice([0, 150, 200], size=10)

#Inconsistent Text Formatting
df["Department"] = df["Department"].astype(str)
# lowercase only the sampled rows (use 'frac' not 'fract')
dept_idx = df.sample(frac=0.1).index
df.loc[dept_idx, "Department"] = df.loc[dept_idx, "Department"].str.lower()


In [9]:
df

,Student_ID,Age,Marks,Department
0,101,NaN,82.0,nan
1,102,21.0,NaN,nan
2,103,NaN,NaN,nan
3,104,NaN,68.0,nan
4,105,NaN,NaN,nan
...,...,...,...,...
1035,263,NaN,75.0,nan
1036,410,NaN,NaN,nan
1037,654,NaN,NaN,nan
1038,203,NaN,NaN,nan


In [10]:
df=df.copy()

In [15]:
# Standardize Column Names  
df.columns =(
     df.columns.str.strip()  # Remove leading/trailing spaces
    .str.lower()           # Convert to lowercase
    .str.replace(' ', '_') # Replace spaces with underscores    
)

In [12]:
df.columns

Index(['student_id', 'age', 'marks', 'department'], dtype='object')

In [17]:
import re
# Enforcing Correct Data Types
# normalize messy column names (remove non-alnum chars, collapse underscores, lowercase)
df.columns = (
	df.columns.astype(str)
	.str.replace(r'[^0-9a-zA-Z_]', '', regex=True)
	.str.replace(r'_+', '_', regex=True)
	.str.strip('_')
	.str.lower()
)

# map fuzzy names to canonical names
col_map = {}
for c in df.columns:
	key = re.sub(r'[^a-z]', '', c.lower())
	if 'age' in key:
		col_map[c] = 'age'
	elif 'mark' in key:
		col_map[c] = 'marks'
	elif 'student' in key:
		col_map[c] = 'student_id'
	elif 'depart' in key:
		col_map[c] = 'department'
if col_map:
	df = df.rename(columns=col_map)

df["age"] = pd.to_numeric(df["age"], errors='coerce')
df["marks"] = pd.to_numeric(df["marks"], errors='coerce')


In [18]:
# Handling Missing Values
df["age"] = df["age"].fillna(df["age"].median(), inplace=True)

In [19]:
#Handling Duplicates
df = df.drop_duplicates(inplace=True)